# CS 893 Advanced Computer Vision (Assignment No 01)

This assignment deals with classification of 10 different traffic signs from dataset which is a subset of Belgium TSC. Two different approaches have been implemented for this classification task. For both implemented algorithms, a pipeline of Computer Vision and Machine Learning algorithms have been used.

# Approach 1 (Key Point Feature Extraction)

Important points of this approach are following:-
1. Read and store the Image dataset
    i.Convert images into grayscale
    ii.Resize images into 128x128 dimensions
2. Extract keypoints and feature vectors using SIFT
3. Segregating these feature vectors into clusters using Kmeans 
4. Assigning feature vectors to respective clusters to form bag of visual words
5. Apply feature scaling
6. Training a multiclass SVM classifier with bag of visual words
7. Making predictions of test dataset and calculating metrics

# Approach 2 (Histogram of Gradients)

Important points of this approach are following:-
1. Read and store the Image dataset
    i.Convert images into grayscale
    ii.Resize images into 128x128 dimensions
2. Extract feature vectors through Histogram of Gradients on the grayscale image
3. Apply feature scaling
4. Apply PCA on scaled features for dimentionality reduction
5. Training a multiclass SVM classifier with the HOG features (scaled and reduced)
6. Making predictions of test dataset and calculating metrics

# Install requisite packages

In [ ]:
! pip uninstall opencv-python -y
! pip uninstall opencv-contrib-python -y
! pip install opencv-python==3.4.11.45
! pip install opencv-contrib-python==3.4.11.45
! pip install scikit-learn
! pip install scikit-image
! pip install matplotlib

# Import Requisite Modules 

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from skimage.feature import hog
from sklearn.decomposition import PCA
import random
from skimage import io

%matplotlib inline

# Supporting Functions

Number of supporting functions have been implemented that are required to successfully train and test both models

In [ ]:
# This function returns the image class name based on the folder name
def FindClass(folderName):
    if folderName == "00001":
        label = "Hump"
    elif folderName == "00002":
        label = "Limit 70"
    elif folderName == "00003":
        label = "Limit 50"
    elif folderName == "00004":
        label = "Children crossing"
    elif folderName == "00005":
        label = "Forbidden Direction"
    elif folderName == "00006":
        label = "Give way"
    elif folderName == "00007":
        label = "No entry"
    elif folderName == "00008":
        label = "Bicycle way"
    elif folderName == "00009":
        label = "Bicyle & children way"
    else:
        label = "No Parking"
    return label
# This function is used for transforming the dataset (Grayscale,Resize) into required format for further processing 
def BuildDataset(datasetDirectoryName, trainFlag = True):
    datasetX = []
    labelsY = []
    folderNamesList = os.listdir(datasetDirectoryName)
    # Loop through all the subfolders of Dataset
    for index,folderName in enumerate(folderNamesList):
        imgClassFolderPath = os.path.join(datasetDirectoryName, folderName)
        if trainFlag:
            filePath = os.path.join(imgClassFolderPath, 'train.txt')
        else:
            filePath = os.path.join(imgClassFolderPath, 'test.txt')

        file = open (filePath, 'r')
        readFile = file.readlines() 
       #loop through all the image files for training/testing in the subfolder (Individual classes)
        for i in range(len(readFile)):
            imgPath = os.path.join(imgClassFolderPath, readFile[i].rstrip("\n"))
            img = cv2.imread(imgPath)
            imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # convert image to grayscale
            imgGray = cv2.resize(imgGray, (128,128))
            datasetX.append(imgGray)
            label = FindClass(folderName)
            labelsY.append(label) 
    return datasetX, labelsY

def ReadDataset(datasetDirectoryName, trainFlag = True):
    datasetX = []
    labelsY = []
    folderNamesList = os.listdir(datasetDirectoryName)
    # Loop through all the subfolders of Dataset
    for index,folderName in enumerate(folderNamesList):
        imgClassFolderPath = os.path.join(datasetDirectoryName, folderName)
        if trainFlag:
            filePath = os.path.join(imgClassFolderPath, 'train.txt')
        else:
            filePath = os.path.join(imgClassFolderPath, 'test.txt')

        file = open (filePath, 'r')
        readFile = file.readlines() 
       #loop through all the image files for training/testing in the subfolder (Individual classes)
        for i in range(len(readFile)):
            imgPath = os.path.join(imgClassFolderPath, readFile[i].rstrip("\n"))
            img = cv2.imread(imgPath)
            datasetX.append(img)
            label = FindClass(folderName)
            labelsY.append(label) 
    return datasetX, labelsY

# This function is used for extracting features from images 
def ExtractFeatures(featureExtractorType, img):
    if featureExtractorType == "SIFT":
        sift = cv2.xfeatures2d.SIFT_create()
        keyPoint, featDes = sift.detectAndCompute(img, None)
    elif featureExtractorType == "HOG":
        featDes = hog(img, orientations=9, pixels_per_cell=(32, 32), cells_per_block=(4, 4), block_norm = 'L2-Hys')
        featDes = np.hstack(featDes)
    return featDes
# This function returns feature vector length based of feature extractor type
def GetFeatureVectorLength(featureExtractorType):
    if featureExtractorType == "SIFT":
        featureVectorLength = 128
    elif featureExtractorType == "HOG":
        featureVectorLength = 1
    return featureVectorLength
# This function is used for stacking features vertically by opening the feature descriptor list along its second dimension
def StackFeatureDescriptors(featDescList):
    featDescriptorStack = []
    for i in range(len(featDescList)):
        featDescriptorStack[i*featDescList[i].shape[0]:featDescList[i].shape[0]-1] = featDescList[i]
    featDescriptorStack = np.array(featDescriptorStack)
    return featDescriptorStack
# This function builds clusters using Kmeans clustering 
def ClusterofFeatures(featDescriptorStack, noOfClusters):
    kmeans = KMeans(n_clusters = noOfClusters, max_iter = 4000, random_state=15)
    kmeans.fit(featDescriptorStack)
    return kmeans
# This function assigns each feature vector from dataset to the clusters (formed during Kmeans cluster building step)
# Bag of Visual Words are created by passing feature vectors from dataset 
# Similar features clustered together
# Each cluster represents a seperate feature type (Visual Word)
def ConstructBOVW(kmeans, featureDescriptorList, datasetLength, noOfClusters, featureVectorLength):
    imgFeatures = np.zeros((datasetLength,noOfClusters))
    for i in range(datasetLength):
        for j in range(len(featureDescriptorList[i])):
            feature = featureDescriptorList[i][j].reshape(1,featureVectorLength)
            clusterNo = kmeans.predict(feature)
            imgFeatures[i][clusterNo] += 1
    return imgFeatures  
# This function is used for plotting the histogram of features in a particular cluster
def PlotHistogram(imgFeatures, noOfClusters):
    clusterAxis = np.arange(1,noOfClusters+1,1)
    featureAxis = np.sum(imgFeatures,axis=0)
    plt.rcParams["figure.figsize"] = (16,8)
    plt.bar(clusterAxis, featureAxis)
    plt.xlabel("Cluster Index")
    plt.ylabel("Frequency")
    plt.title("Histogram of features for the Dataset")
    plt.show()
#This function scales the features by removing the mean = 0 and scaling to unit variance
def ScaleFeatures(imgFeatures):
    scaleObj = StandardScaler().fit(imgFeatures)        
    scaledImgFeatures = scaleObj.transform(imgFeatures)
    return scaleObj, scaledImgFeatures
#This function applies Principal Component Analysis for dimensionality reduction of features
def ApplyPCA(imgFeatures):
    pcaObj = PCA(n_components=50)
    pcaFeatures = pcaObj.fit_transform(imgFeatures)
    return pcaObj, pcaFeatures
#This function selects the optimized hyperparameters for State Vector Machine
def SelectHyperparametersSVM(dataX,labelY):
    print("Starting search for SVM hyperparameters....")
    param_grid = {'C': [0.05, 1, 5, 10, 20, 40],
              'gamma': [0.1 ,0.01, 0.005, 0.001, 0.0005, 0.0001],
              'kernel': ['rbf','poly']}
    grid = GridSearchCV(SVC(), param_grid, refit = True)
    grid.fit(dataX, labelY)
    print("Search for SVM hyperparameters complete")
    return grid.best_params_
#This function trains SVM classifier
def TrainSVM(imgFeatures, labelY):
    params = SelectHyperparametersSVM(imgFeatures, labelY)
    cParam, gammaParam, kernelType = params.get("C"), params.get("gamma"), params.get("kernel")
    print(cParam, gammaParam, kernelType)
    svm = SVC(kernel = kernelType, C =  cParam, gamma = gammaParam, random_state=15)
    svm.fit(imgFeatures, labelY)
    return svm
#This function trains Randomforest classifier
def TrainRandomForestClassifier(ImgFeaturesX,labelsTrainY):
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(ImgFeaturesX,labelsTrainY)
    return clf
#This function is used for printing Confusion Matrix
def DisplayConfusions(labelsY, predictions, classNames):
    confMat = confusion_matrix(labelsY, predictions, labels = classNames)
    print(confMat)
    plt.rcParams["figure.figsize"] = (14,7)
    disp = ConfusionMatrixDisplay(confusion_matrix=confMat, display_labels=classNames)
    disp.plot(xticks_rotation = 90)
    plt.show()  
#This function calculates the accuracy of the trained model for test dataset
def CalculateAccuracy(labelsY, predictions):
    print ('accuracy score: %f' % accuracy_score(labelsY, predictions))
#This function display images    
def DisplayImages(datasetX, index, predictions):
    for i in range(len(index)):
        plt.subplot(5,5,i+1)
        img = cv2.cvtColor(datasetX[index[i]], cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.title(predictions[index[i]])
    plt.show()

# Train Model (Approach 1)

In [ ]:
def TrainModelKP(datasetDirectoryName, noOfClusters, featureExtractorType, classifierType):
    featureVectorLength = GetFeatureVectorLength(featureExtractorType)
    featureDescriptorTrainList = []
    print("Processing dataset....")
    datasetTrainX, labelsTrainY = BuildDataset(datasetDirectoryName, trainFlag = True)
    print("Processing of dataset complete")
    print("Extracting Features.....")
    # Loop through complete dataset, extract feature vectors and stack them to form a feature vector stack 
    for i in range(len(datasetTrainX)):
        featureDescriptorTrainList.append(ExtractFeatures(featureExtractorType, datasetTrainX[i]))
    print("Feature Extraction complete")
    featDescriptorStackTrain = StackFeatureDescriptors(featureDescriptorTrainList)
    print("Feature Descriptor dimensions:"+str(featDescriptorStackTrain.shape))
    print("Kmeans clustering ....")
    kmeans = ClusterofFeatures(featDescriptorStackTrain, noOfClusters)
    print("Kmeans clustering complete")
    print("Constructing bag of visual words....")
    imgFeaturesTrain = ConstructBOVW(kmeans, featureDescriptorTrainList, len(datasetTrainX), noOfClusters, featureVectorLength)
    print("Constructing bag of visual words complete")
    #PlotHistogram(imgFeaturesTrain, noOfClusters)
    scaleObj, scaledImgFeaturesTrain = ScaleFeatures(imgFeaturesTrain)
    print("Training Classifier....")
    if classifierType == 'SVM':
        classifier = TrainSVM(scaledImgFeaturesTrain, labelsTrainY)
    else:
        classifier = TrainClassifier(scaledImgFeaturesTrain,labelsTrainY) # Random Forest Classifier
    print("Training Classifier complete")
    return classifier, scaleObj, kmeans, scaledImgFeaturesTrain

# Train Model (Approach 2)

In [ ]:
def TrainModelHog(datasetDirectoryName, featureExtractorType, classifierType):
    featureDescriptorTrainList = []
    print("Processing dataset....")
    datasetTrainX, labelsTrainY = BuildDataset(datasetDirectoryName, trainFlag = True)
    print("Processing of dataset complete")
    print("No of training images: " + str(len(datasetTrainX)))
    print("Extracting Features.....")
    # Loop through complete dataset, extract feature vectors and stack them to form a feature vector stack 
    for i in range(len(datasetTrainX)):
        featureDescriptorTrainList.append(ExtractFeatures(featureExtractorType, datasetTrainX[i]))
    featureDescriptorTrainArray = np.array(featureDescriptorTrainList)
    print("Feature Extraction complete")
    print("Feature Descriptor dimensions:"+str(featureDescriptorTrainArray.shape))
    scaleObj, scaledImgFeatures = ScaleFeatures(featureDescriptorTrainArray)
    pcaObj, pcaFeatures = ApplyPCA(scaledImgFeatures)
    print("Feature Dimensions after PCA:"+str(pcaFeatures.shape))
    print("Training Classifier....")
    if classifierType == 'SVM':
        classifier = classifier = TrainSVM(pcaFeatures, labelsTrainY)
    else:
        classifier = TrainClassifier(pcaFeatures,labelsTrainY) # Random Forest Classifier
    print("Training of Classifier complete")
    return classifier, scaleObj, pcaObj, pcaFeatures

# Execute Model Training

1. Before executing this cell, import requisite modules and execute the supporting functions.
2. It is also assumed that dataset folder "CS893 Sp2022 A1 Dataset" is present in same folder.
3. For Approach 1 change {featureExtractorType = "SIFT"}
4. For Approach 2 change {featureExtractorType = "HOG"}

In [ ]:
datasetDirectoryName = "CS893 Sp2022 A1 Dataset"
noOfClusters = 500 # No of clusters for Kmeans
featureExtractorType = "HOG" # Options include SIFT, HOG
classifierType = "SVM" # Options include SVM, RANDOMFOREST
if featureExtractorType == "HOG": 
    classifier, scaleObj, pcaObj, imgFeaturesTrain = TrainModelHog(datasetDirectoryName, featureExtractorType, classifierType)
else:
    classifier, scaleObj, kmeans, imgFeaturesTrain = TrainModelKP(datasetDirectoryName, noOfClusters, featureExtractorType, classifierType)

# Testing the model

# Test Model Approach 1

In [ ]:
def TestModelKP(datasetDirectoryName, noOfClusters, featureExtractorType, classifierType):
    featureVectorLength = GetFeatureVectorLength(featureExtractorType)
    featureDescriptorTestList = []
    print("Processing dataset started....")
    datasetTestX, labelsTestY = BuildDataset(datasetDirectoryName, trainFlag = False)
    print("Processing of dataset completed....")
    print("No of test images: " + str(len(datasetTestX)))
    print("Extracting Features.....")
    # Loop through complete dataset, extract feature vectors and stack them to form a feature vector stack 
    for i in range(len(datasetTestX)):
        featureDescriptorTestList.append(ExtractFeatures(featureExtractorType, datasetTestX[i]))
    print("Feature Extraction complete")
    print("Constructing Bag of Visual Words....")
    imgFeaturesTest = ConstructBOVW(kmeans, featureDescriptorTestList, len(datasetTestX), noOfClusters, featureVectorLength)
    print("Constructing Bag of Visual Words complete")
    scaledImgFeaturesTest = scaleObj.transform(imgFeaturesTest)
    print("Predicting on test dataset....")
    if classifierType == 'SVM':
        predictions = classifier.predict(scaledImgFeaturesTest)
    else:
        predictions = classifier.predict(scaledImgFeaturesTest) # Random Forest Classifier
    return labelsTestY, predictions

# Test Model Approach 2

In [ ]:
def TestModelHOG(datasetDirectoryName, featureExtractorType, classifierType):
    featureDescriptorTestList = []
    print("Processing dataset started....")
    datasetTestX, labelsTestY = BuildDataset(datasetDirectoryName, trainFlag = False)
    print("Processing of dataset completed")
    print("No of test images: " + str(len(datasetTestX)))
    print("Extracting Features.....")
    # Loop through complete dataset, extract feature vectors and stack them to form a feature vector stack 
    for i in range(len(datasetTestX)):
        featureDescriptorTestList.append(ExtractFeatures(featureExtractorType, datasetTestX[i]))
    
    featureDescriptorTestArray = np.array(featureDescriptorTestList)
    print("Feature Extraction complete")
    print("Feature Descriptor dimensions:"+str(featureDescriptorTestArray.shape))
    scaledImgFeaturesTest = scaleObj.transform(featureDescriptorTestArray)
    pcaFeaturesTest = pcaObj.transform(scaledImgFeaturesTest)
    print("Feature Dimensions after PCA:"+str(pcaFeaturesTest.shape))
    print("Predicting on test dataset....")
    if classifierType == 'SVM':
        predictions = classifier.predict(pcaFeaturesTest)
    else:
        predictions = classifier.predict(pcaFeaturesTest) # Random Forest Classifier
    return labelsTestY, predictions

# Execute Model Testing

1. Before executing this cell, make sure that training for respective model has been executed.
2. It is also assumed that dataset folder "CS893 Sp2022 A1 Dataset" is present in same folder.
3. For Approach 1 change {featureExtractorType = "SIFT"}
4. For Approach 2 change {featureExtractorType = "HOG"}

In [ ]:
classNames = ["Hump", "Limit 70", "Limit 50", "Children crossing", "Forbidden Direction", "Give way", "No entry", "Bicycle way", "Bicyle & children way", "No Parking"]
datasetDirectoryName = "CS893 Sp2022 A1 Dataset"
noOfClusters = 500
featureExtractorType = "HOG"
classifierType = "SVM"
if featureExtractorType == "HOG":
    labelsTestY, predictions = TestModelHOG(datasetDirectoryName, featureExtractorType, classifierType)
else:
    labelsTestY, predictions = TestModelKP(datasetDirectoryName, noOfClusters, featureExtractorType, classifierType)
print("Confusion Matrix for test dataset")
labelsTestY = np.array(labelsTestY)
DisplayConfusions(labelsTestY, predictions, classNames)
CalculateAccuracy(labelsTestY, predictions)
classificationReport = classification_report(labelsTestY,predictions,labels=classNames)
print(classificationReport)

# Print Wrongly Classified Images

In [ ]:
a = labelsTestY==predictions
b = np.where(a==False)
wronglyClassifiedIndexes = b[0]

datasetX, labelsY = ReadDataset(datasetDirectoryName, trainFlag = False)
print("Wronly Classified Images")
DisplayImages(datasetX, wronglyClassifiedIndexes, predictions)